# 🧩 Módulo 7 — Pandas: Agrupaciones y `groupby`

En este notebook aprenderás a hacer análisis avanzado con `groupby`, incluyendo:

- Agrupaciones básicas
- Múltiples agregaciones
- Transformaciones
- Filtrado de grupos
- Tablas dinámicas (`pivot_table`)
- Multiíndice

---

## 1️⃣ Dataset de ejemplo

In [4]:
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'categoria': ['A','A','B','B','B','C','C'],
    'producto': ['p1','p2','p3','p4','p5','p6','p7'],
    'ventas': [100, 150, 200, 50, 120, 80, 60],
    'coste': [60, 90, 120, 30, 70, 40, 20]
})
df

,categoria,producto,ventas,coste
0,A,p1,100,60
1,A,p2,150,90
2,B,p3,200,120
3,B,p4,50,30
4,B,p5,120,70
5,C,p6,80,40
6,C,p7,60,20


---
## 2️⃣ Agrupaciones básicas

Ventas totales por categoría:

In [5]:
df.groupby('categoria')['ventas'].sum()

categoria
A    250
B    370
C    140
Name: ventas, dtype: int64

Promedio de ventas por categoría:

In [6]:
df.groupby('categoria')['ventas'].mean()

categoria
A    125.000000
B    123.333333
C     70.000000
Name: ventas, dtype: float64

---
## 3️⃣ Agrupaciones múltiples

Obtener estadísticas combinadas:

In [7]:
df.groupby('categoria').agg({'ventas':['sum','mean'], 'coste':['sum','mean']})

ventas             coste           
             sum        mean   sum       mean
categoria                                    
A            250  125.000000   150  75.000000
B            370  123.333333   220  73.333333
C            140   70.000000    60  30.000000

Resumen estadístico por grupo:

In [8]:
df.groupby('categoria')['ventas'].describe()

,count,mean,std,min,25%,50%,75%,max
categoria,,,,,,,,
A,2.0,125.000000,35.355339,100.0,112.5,125.0,137.5,150.0
B,3.0,123.333333,75.055535,50.0,85.0,120.0,160.0,200.0
C,2.0,70.000000,14.142136,60.0,65.0,70.0,75.0,80.0


---
## 4️⃣ Transformaciones (`transform`)

Calcular ventas normalizadas por categoría:

In [9]:
df['ventas_norm'] = df.groupby('categoria')['ventas'].transform(lambda x: (x - x.mean())/x.std())
df[['categoria','producto','ventas','ventas_norm']]

,categoria,producto,ventas,ventas_norm
0,A,p1,100,-0.707107
1,A,p2,150,0.707107
2,B,p3,200,1.021466
3,B,p4,50,-0.977054
4,B,p5,120,-0.044412
5,C,p6,80,0.707107
6,C,p7,60,-0.707107


---
## 5️⃣ Filtrado de grupos (`filter`)

Ejemplo: seleccionar solo categorías con ventas totales ≥ 200:

In [13]:
def muestra(X):
    print(X)
    return True
df.groupby('categoria').filter(muestra)

  categoria producto  ventas  coste  ventas_norm
0         A       p1     100     60    -0.707107
1         A       p2     150     90     0.707107
  categoria producto  ventas  coste  ventas_norm
2         B       p3     200    120     1.021466
3         B       p4      50     30    -0.977054
4         B       p5     120     70    -0.044412
  categoria producto  ventas  coste  ventas_norm
5         C       p6      80     40     0.707107
6         C       p7      60     20    -0.707107


,categoria,producto,ventas,coste,ventas_norm
0,A,p1,100,60,-0.707107
1,A,p2,150,90,0.707107
2,B,p3,200,120,1.021466
3,B,p4,50,30,-0.977054
4,B,p5,120,70,-0.044412
5,C,p6,80,40,0.707107
6,C,p7,60,20,-0.707107


In [ ]:
# filtro y me quedo con los grupos que tienen una venta superior a 200 por categoria
df.groupby('categoria').filter(lambda g: g['ventas'].sum() >= 200)

,categoria,producto,ventas,coste,ventas_norm
0,A,p1,100,60,-0.707107
1,A,p2,150,90,0.707107
2,B,p3,200,120,1.021466
3,B,p4,50,30,-0.977054
4,B,p5,120,70,-0.044412


In [ ]:
# vuelvo a agrupar para sumar las ventas de los grupos filtrados
df.groupby('categoria').filter(lambda g: g['ventas'].sum() >= 200).groupby('categoria')['ventas'].sum()

categoria
A    250
B    370
Name: ventas, dtype: int64

---
## 6️⃣ Tablas dinámicas (pivot table)


In [ ]:
# obtenemos una vista a la anterior, como tabla dinamia -> es una manera sencillad de agrupar
pd.pivot_table(df, values='ventas', index='categoria', aggfunc='sum')

,ventas
categoria,
A,250
B,370
C,140


Pivot table con varias funciones:

In [17]:
pd.pivot_table(df, values=['ventas','coste'], index='categoria', aggfunc=['sum','mean'])

sum              mean            
          coste ventas      coste      ventas
categoria                                    
A           150    250  75.000000  125.000000
B           220    370  73.333333  123.333333
C            60    140  30.000000   70.000000

---
## 7️⃣ multi-índice

Agrupar por categoría y producto:

In [18]:
multi = df.groupby(['categoria','producto'])['ventas'].sum()
multi

categoria  producto
A          p1          100
           p2          150
B          p3          200
           p4           50
           p5          120
C          p6           80
           p7           60
Name: ventas, dtype: int64

Convertir multi-índice en columnas (reset index):

In [19]:
multi.reset_index()

,categoria,producto,ventas
0,A,p1,100
1,A,p2,150
2,B,p3,200
3,B,p4,50
4,B,p5,120
5,C,p6,80
6,C,p7,60


---
## 8️⃣ Ejercicio práctico

Usando `df`:

### 🧩 Objetivos
1. Calcula ventas totales por categoría
2. Calcula el margen como `ventas - coste` y obtén media por categoría
3. Normaliza `coste` por categoría usando `transform`
4. Haz un `filter` para quedarte solo con categorías cuyo *promedio de ventas* > 100
5. Construye una `pivot_table` que muestre el total de ventas y coste por categoría

Escribe tu solución aquí:

In [ ]:
import pandas as pd
import numpy as np

dfEj = pd.DataFrame({
    'categoria': ['A','A','B','B','B','C','C'],
    'producto': ['p1','p2','p3','p4','p5','p6','p7'],
    'ventas': [100, 150, 200, 50, 120, 80, 60],
    'coste': [60, 90, 120, 30, 70, 40, 20]
})

print("//-- Datos ini: --//")
print(dfEj)
print()


//-- Datos ini: --//
  categoria producto  ventas  coste
0         A       p1     100     60
1         A       p2     150     90
2         B       p3     200    120
3         B       p4      50     30
4         B       p5     120     70
5         C       p6      80     40
6         C       p7      60     20


---
## ✅ Solución (oculta)
<details>
<summary>Mostrar soluciones</summary>

```python
df.groupby('categoria')['ventas'].sum()
```

```python
df['margen'] = df['ventas'] - df['coste']
df.groupby('categoria')['margen'].mean()
```

```python
df['coste_norm'] = df.groupby('categoria')['coste'].transform(lambda x: (x-x.mean())/x.std())
```

```python
df.groupby('categoria').filter(lambda g: g['ventas'].mean() > 100)
```

```python
pd.pivot_table(df, values=['ventas','coste'], index='categoria', aggfunc='sum')
```
</details>